In [1]:
import numpy as np
import os
import pandas as pd
import csv
from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, BatchNormalization, Activation, MaxPool2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skimage import io
from skimage.transform import resize
import tensorflow as tf

Using TensorFlow backend.


In [8]:
path = os.getcwd() + '/histopathologic-cancer-detection'
print(path)

/home/arkadiusz/Documents/Dokumenty/Projekt NN/histopathologic-cancer-detection


In [12]:
files_list = os.listdir(path+'/train')[:60000]

['85a02c5fab14b36a5fe611bcd6e6b43307bf0a6a.tif',
 '35fa6fc6712723765cafbf3cbb8245a6d696b7b8.tif',
 '3ab0814d714de43efac1adda38396f3fc729fd0a.tif',
 '87372d25911f98c04cc41aa8b7649309ab216fe2.tif',
 '25202670bf31befa859e4c66a13e5f8a624242e3.tif',
 '165e0a899e0ce433abf652ccbbc0540dc94cd2e3.tif',
 '4ad902f8d215faab70098b01af2ced1eb689b68b.tif',
 '8885d688e818c9f986036b909f5b6383c2576e23.tif',
 'a9f10a859c7510dbfdf1475df3d8d9b88a2725cb.tif',
 'd7ca740e9bdfa38175c00540b7dc10536056db87.tif',
 'c3b03624a7dc91fb4cb0a52e3320ed24fa2ada81.tif',
 'b8a1e9c88d3a7648be39e959c20b8db40d2ae5c6.tif',
 '0ee5c39382354c96e68deb4c7379ff3a1c7e90a9.tif',
 '404cb6fae89f87c58381fb3f7f03d662f7a1b03e.tif',
 '782b62e4d85944b1f65d1e84e30b22e4ef8a707d.tif',
 '15bd6dadf21bd99bb6127753d8b67e53d421e82a.tif',
 '24a5156ee1ff5d1fb32d1f8f8450d81d57f5829c.tif',
 'c78a9fb3d96d2a18b67b4402875a816e7cdfe962.tif',
 'c41caa2549b4ffaccf665103a334036bd83de341.tif',
 '191f6010874cc6cc51223c736f717a489ef4560b.tif',
 '64f1e179f9f993f5c6

In [14]:
labels = {}
with open(path+'/train_labels.csv','r') as f:
    next(f)
    lines = csv.reader(f)
    for i in lines:
        labels[i[0]] = int(i[1])



train = []
os.chdir('histopathologic-cancer-detection/train')
for file in files_list:
    #bw = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    train.append([io.imread(file, plugin='tifffile'),labels[file.split('.')[0]]])

xtrain = []
ytrain = []
for i,j in train:
	xtrain.append(i)
	ytrain.append(j)

train_images = np.array(xtrain).astype('float32')
train_images /= 255

train_labels = np.array(ytrain)
train_labels = np_utils.to_categorical(train_labels,2)


In [8]:
first_filters=32
second_filters=64
third_filters=128
kernel_size=(3,3)
pool_size=(2,2)
dropout_conv = 0.3
dropout_dense = 0.5

model=Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (96, 96, 3)))
model.add(Conv2D(first_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

#model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(256, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(2, activation = "sigmoid"))

model.compile(optimizer='Adam', loss = "categorical_crossentropy", metrics=["accuracy"])

In [12]:
model = Sequential()

model.add(Conv2D(32,(3,3), activation='relu', input_shape=(96,96,3)))
model.add(Conv2D(32,(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])        

In [13]:
model.fit(train_images, train_labels, batch_size=32, validation_split=0.2, epochs=10)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 79s 2ms/step - loss: 0.4905 - acc: 0.7707 - val_loss: 0.4546 - val_acc: 0.7913
Epoch 2/10
48000/48000 [==============================] - 78s 2ms/step - loss: 0.4262 - acc: 0.8116 - val_loss: 0.4033 - val_acc: 0.8195
Epoch 3/10
48000/48000 [==============================] - 79s 2ms/step - loss: 0.3795 - acc: 0.8377 - val_loss: 0.3648 - val_acc: 0.8457
Epoch 4/10
48000/48000 [==============================] - 77s 2ms/step - loss: 0.3418 - acc: 0.8532 - val_loss: 0.3329 - val_acc: 0.8542
Epoch 5/10
48000/48000 [==============================] - 78s 2ms/step - loss: 0.3124 - acc: 0.8667 - val_loss: 0.3251 - val_acc: 0.8592
Epoch 6/10
48000/48000 [==============================] - 78s 2ms/step - loss: 0.2912 - acc: 0.8776 - val_loss: 0.3320 - val_acc: 0.8594
Epoch 7/10
48000/48000 [==============================] - 77s 2ms/step - loss: 0.2649 - acc: 0.8910 - val_loss: 0.3360 - val_acc

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 94, 94, 32)        896       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 92, 92, 32)        9216      
_________________________________________________________________
batch_normalization_15 (Batc (None, 92, 92, 32)        128       
_________________________________________________________________
activation_14 (Activation)   (None, 92, 92, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 46, 46, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 46, 46, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 44, 44, 64)        18432     
__________